In [ ]:
!pip install torch --index-url https://download.pytorch.org/whl/cu121
!pip install transformers
!pip install datasets

Looking in indexes: https://download.pytorch.org/whl/cu121
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
import json
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# Load dataset from uploaded file
dataset_path = "gradingnew.json"

with open(dataset_path, "r") as file:
    dataset = json.load(file)

# Convert dataset into text-to-text format
data = []
for item in dataset:
    question = item["question"]
    for ans in item["answers"]:
        input_text = f"Grade the following answer:\nQuestion: {question}\nStudent Answer: {ans['text']}"
        target_text = f"{ans['score']} | {ans['feedback']}"
        data.append({"input_text": input_text, "target_text": target_text})

# Split into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_dict({
    "input_text": [x["input_text"] for x in train_data],
    "target_text": [x["target_text"] for x in train_data],
})

val_dataset = Dataset.from_dict({
    "input_text": [x["input_text"] for x in val_data],
    "target_text": [x["target_text"] for x in val_data],
})

dataset_dict = DatasetDict({"train": train_dataset, "validation": val_dataset})

# Load T5 Tokenizer and Model
model_name = "t5-small"  # You can change to "t5-base" if needed
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Tokenization function
def preprocess_data(examples):
    inputs = tokenizer(examples["input_text"], padding="max_length", truncation=True, max_length=512)
    targets = tokenizer(examples["target_text"], padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = targets["input_ids"]
    return inputs

# Apply tokenization
tokenized_datasets = dataset_dict.map(preprocess_data, batched=True)

# Debug: Check if tokenized dataset has proper inputs
print("\n🔹 Sample Tokenized Input:", tokenizer.decode(tokenized_datasets["train"][0]["input_ids"]))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Map:   0%|          | 0/462 [00:00<?, ? examples/s]

Map:   0%|          | 0/116 [00:00<?, ? examples/s]


🔹 Sample Tokenized Input: Grade the following answer: Question: What is the purpose of the pandas library in Python? Student Answer: Pandas is just a faster version of lists in Python.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [ ]:
training_args = TrainingArguments(
    output_dir="./grading_model",
    evaluation_strategy="epoch",  # Enable evaluation
    save_strategy="epoch",
    learning_rate=2e-4,  # Reduce learning rate for better training stability
    per_device_train_batch_size=2,  # Reduce batch size for less memory usage
    per_device_eval_batch_size=2,
    num_train_epochs=5,  # Increase epochs for better learning
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
)

# Train the Model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)
trainer.train()

# Save Fine-Tuned Model
trainer.save_model("./grading_model")
tokenizer.save_pretrained("./grading_model")

print("✅ Training completed! Model saved at ./grading_model")

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-6-bdbbc05e22ef>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.194000,0.179673
2,0.156700,0.163288
3,0.192300,0.155276
4,0.088800,0.155302
5,0.091600,0.155070


✅ Training completed! Model saved at ./grading_model


In [ ]:
# Load trained model and tokenizer
model_path = "./grading_model"  # Update path if needed
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

def evaluate_answer(question, student_answer):
    """Passes input through the model and decodes output"""
    input_text = f"Grade the following answer:\nQuestion: {question}\nStudent Answer: {student_answer}"
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

    # Generate output
    outputs = model.generate(**inputs, max_length=50)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return result

# Define test cases
test_cases = [
    ("What is the difference between a list and a tuple in Python?",
     "A list is mutable, while a tuple is immutable. Lists are slower than tuples."),

    ("What is the difference between a list and a tuple in Python?",
     "Lists are changeable, but tuples are not. Lists use square brackets, tuples use parentheses."),

    ("What is the difference between a list and a tuple in Python?",
     "Lists and tuples are the same and can be modified."),

    ("What is the difference between a list and a tuple in Python?",
     "Lists and tuples store data. Lists are mutable, meaning they can be changed, while tuples are immutable. Lists use square brackets, whereas tuples use parentheses. Tuples are slightly faster."),

    ("What is the difference between a list and a tuple in Python?",
     "Lists can change, tuples can't.")
]

# Run tests
print("🔹 Model Test Results:")
for i, (question, answer) in enumerate(test_cases):
    result = evaluate_answer(question, answer)
    print(f"\nTest {i+1}:")
    print(f"📝 Student Answer: {answer}")
    print(f"🔹 Model Output: {result}")

🔹 Model Test Results:

Test 1:
📝 Student Answer: A list is mutable, while a tuple is immutable. Lists are slower than tuples.
🔹 Model Output: 3 | Partially correct, but missing some key aspects.

Test 2:
📝 Student Answer: Lists are changeable, but tuples are not. Lists use square brackets, tuples use parentheses.
🔹 Model Output: 3 | Partially correct, but missing some key aspects.

Test 3:
📝 Student Answer: Lists and tuples are the same and can be modified.
🔹 Model Output: 3 | Partially correct, but missing some key aspects.

Test 4:
📝 Student Answer: Lists and tuples store data. Lists are mutable, meaning they can be changed, while tuples are immutable. Lists use square brackets, whereas tuples use parentheses. Tuples are slightly faster.
🔹 Model Output: 5 | Excellent! Well-explained with clarity.

Test 5:
📝 Student Answer: Lists can change, tuples can't.
🔹 Model Output: 3 | Partially correct, but missing some key aspects.


In [ ]:
test_cases = [
    ("What are instance and class variables in Python?",
     "Instance variables are stored in a function, class variables are not."),

    ("What is the difference between a list and a tuple in Python?",
     "Lists are changeable, but tuples are not. Lists use square brackets, tuples use parentheses."),

    ("What is k-means clustering?",
     "Lists and tuples are the same and can be modified."),

    ("What is k-means clustering?",
     "It is a clustering algorithm that forms clusters based on similarity in data points."),

    ("What is k-means clustering?",
     "K-means clustering is a supervised learning algorithm used for classification problems.")
]

# Run tests
print("🔹 Model Test Results:")
for i, (question, answer) in enumerate(test_cases):
    result = evaluate_answer(question, answer)
    print(f"\nTest {i+1}:")
    print(f"📝 Student Answer: {answer}")
    print(f"🔹 Model Output: {result}")

🔹 Model Test Results:

Test 1:
📝 Student Answer: Instance variables are stored in a function, class variables are not.
🔹 Model Output: 3 | Partially correct, but missing some key aspects.

Test 2:
📝 Student Answer: Lists are changeable, but tuples are not. Lists use square brackets, tuples use parentheses.
🔹 Model Output: 3 | Partially correct, but missing some key aspects.

Test 3:
📝 Student Answer: Lists and tuples are the same and can be modified.
🔹 Model Output: 5 | Excellent! Well-explained with clarity.

Test 4:
📝 Student Answer: It is a clustering algorithm that forms clusters based on similarity in data points.
🔹 Model Output: 5 | Excellent! Well-explained with clarity.

Test 5:
📝 Student Answer: K-means clustering is a supervised learning algorithm used for classification problems.
🔹 Model Output: 5 | Excellent! Well-explained with clarity.


In [ ]:
!pip install evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.4 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=b5d7ef3717941dbfd9e63376d002434ac2450f0c693df8150fe2a9b3064cf960
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
import evaluate
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import Dataset

# Load trained model and tokenizer
model_path = "./grading_model"  # Update path if needed
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

# Load the ROUGE metric correctly
rouge_metric = evaluate.load("rouge")

# Define evaluation function
def evaluate_model(model, tokenizer, dataset):
    predictions = []
    references = []

    for example in dataset:
        input_text = example["input"]
        expected_output = example["output"]  # True label

        # Tokenize input
        input_ids = tokenizer(input_text, return_tensors="pt").input_ids

        # Generate prediction
        output_ids = model.generate(input_ids, max_length=128)
        generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        predictions.append(generated_text)
        references.append(expected_output)

    # Compute ROUGE scores
    rouge = rouge_metric.compute(predictions=predictions, references=references)

    return {"ROUGE-L Score": rouge["rougeL"]}  # ✅ Fix: Remove `.mid.fmeasure`

# Load validation dataset (ensure it’s properly formatted)
import json

with open("/content/gradingnew.json", "r") as file:
    dataset_json = json.load(file)

# Convert dataset into structured format
val_data = []
for item in dataset_json:
    question = item["question"]
    for answer_entry in item["answers"]:
        val_data.append({
            "input": f"Question: {question} \nAnswer: {answer_entry['text']}",
            "output": f"Score: {answer_entry['score']} \nFeedback: {answer_entry['feedback']}"
        })

# Convert to Hugging Face Dataset
val_dataset = Dataset.from_list(val_data)

# Run Evaluation
results = evaluate_model(model, tokenizer, val_dataset)

print("📊 Evaluation Results:")
print(f"ROUGE-L Score: {results['ROUGE-L Score']:.4f}")  # ✅ ROUGE for grading & feedback accuracy


📊 Evaluation Results:
ROUGE-L Score: 0.2318


In [ ]:
from transformers import pipeline
import json

# Load dataset
with open("/content/gradingnew.json", "r") as file:
    dataset = json.load(file)

# Load T5 paraphraser model
paraphraser = pipeline("text2text-generation", model="t5-small")

# Paraphrase function
def paraphrase_text(text):
    return paraphraser(f"Paraphrase: {text}", max_length=100, num_return_sequences=1)[0]["generated_text"]

# Apply augmentation
augmented_data = []
for item in dataset:
    original_question = item["question"]
    paraphrased_question = paraphrase_text(original_question)

    new_answers = []
    for answer_entry in item["answers"]:
        paraphrased_answer = paraphrase_text(answer_entry["text"])
        new_answers.append({
            "text": paraphrased_answer,
            "score": answer_entry["score"],  # Keep score the same
            "feedback": answer_entry["feedback"]  # Keep feedback the same
        })

    # ✅ Append paraphrased question as a new entry (original stays)
    augmented_data.append({"question": paraphrased_question, "answers": new_answers})

# ✅ Combine original dataset + augmented dataset
dataset.extend(augmented_data)

# Save the new dataset
with open("/content/augmented_grading.json", "w") as file:
    json.dump(dataset, file, indent=4)

print("✅ Data augmentation complete! Original + Paraphrased entries retained.")


Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


✅ Data augmentation complete! Original + Paraphrased entries retained.


In [ ]:
def paraphrase_text(text):
    # Generate paraphrase
    outputs = paraphraser(text, max_length=100, num_return_sequences=1)

    # Extract only the generated text (remove prompt-like output)
    return outputs[0]["generated_text"].replace("Paraphrase: ", "").strip()

# Apply augmentation
augmented_data = []
for item in dataset:
    original_question = item["question"]
    paraphrased_question = paraphrase_text(original_question)  # ✅ Fixed function used

    new_answers = []
    for answer_entry in item["answers"]:
        paraphrased_answer = paraphrase_text(answer_entry["text"])  # ✅ Fixed function used
        new_answers.append({
            "text": paraphrased_answer,
            "score": answer_entry["score"],  # Keep original grading
            "feedback": answer_entry["feedback"]  # Keep original feedback
        })

    # ✅ Append the fixed paraphrased question and answers as a new entry
    augmented_data.append({"question": paraphrased_question, "answers": new_answers})

# ✅ Combine original + augmented data
dataset.extend(augmented_data)

# Save the new dataset
with open("/content/augmented_grading1.json", "w") as file:
    json.dump(dataset, file, indent=4)

print("✅ Data augmentation complete! No extra 'Paraphrase:' text in output.")


✅ Data augmentation complete! No extra 'Paraphrase:' text in output.


In [ ]:
!pip install peft